#**Produção de Raio X**

Nomes: Caio, Mayara e Victória

**Objetivo do experimento:**



1.   Determinar o espectro de emissão de RX do tubo de Mo (intensidade x Energia): radiação característica e de freamento.
2.   Determinar a constante de Planck a partir do fenômeno de radiação de freamento.
3.   Avaliar a relação entre θmin e Emax.

**Observações sobre os dados:**


*   O primeiro conjunto de dados é formado por 3 subconjuntos de dados, tomados no intervalo de ângulos de θmin = 2.5̊  e θmax = 30̊ , Δt = 1s, Δθ = 0.1̊ , I = 1 mA, U = 35 kV.
*   O segundo conjunto de dados é formado por 10 subconjuntos de dados, em que temos 5 valores de tensão diferentes e para cada valor de tensão temos dois valores de Δt: Δt = 1s e Δt = 5s. Todos foram obtidos no intervalo de ângulo de θmin = 2.5̊
e θmax = 12̊ . Os valores de tensão foram: 18 kV, 25 kV, 28 kV, 30 kV e 35 kV. A corrente I = 1 mA e Δθ = 0.1̊.

**Observações sobre o experimento:**



*   A incerteza na energia máxima (θmin) é de 1%.
*   A incerteza do equipamento em θ é de 0.05̊.
*   O software do equipamento entrega a coluna de contagens divididas por Δt, então no caso dos conjuntos em que Δt = 5s temos que multiplicar a coluna de contagem por 5 para depois acrescentar a influência do tempo no cáluclo da incerteza: σ(N) = $\sqrt{N}$/Δt








**Importando bibliotecas**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.stats import linregress as lr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Lendo os datasets**

In [ ]:
def read_c1(num):
  num = str(num)
  d1 = np.loadtxt(f'/content/drive/MyDrive/Lab 5/Raio X/Dados/c1-{num}.txt', skiprows=1, dtype=np.str)
  d2 = np.char.replace(d1, ',', '.').astype(np.float64)
  return pd.DataFrame(d2, columns=['\u03B8 (°)','Count'])

def read_c2(tensao, tempo):
  tensao = str(tensao)
  tempo = str(tempo)
  d1 = np.loadtxt(f'/content/drive/MyDrive/Lab 5/Raio X/Dados/c2-{tensao}-{tempo}.txt', skiprows=1, dtype=np.str)
  d2 = np.char.replace(d1, ',', '.').astype(np.float64)
  d3 = pd.DataFrame(d2, columns=['\u03B8 (°)','Count'])
  if tempo == '5':
    d3['Count'] = d3['Count']*5
  return d3
 

  

#Conjunto 1:
c1_1 = read_c1(1)
c1_2 = read_c1(2)
c1_3 = read_c1(3)

#Conjunto 2:
c2_18_1 = read_c2(18,1)
c2_18_5 = read_c2(18,5)

c2_25_1 = read_c2(25,1)
c2_25_5 = read_c2(25,5)

c2_28_1 = read_c2(28,1)
c2_28_5 = read_c2(28,5)

c2_30_1 = read_c2(30,1)
c2_30_5 = read_c2(30,5)

c2_35_1 = read_c2(35,1)
c2_35_5 = read_c2(35,5)

In [ ]:
#Checando os datasets
c1_1

#**Conjunto de dados 1:**



*   Comentar qualitativamente o espectro observado (espectro contínuo e linhas características).
*   Qual critério usar para avaliar as incertezas experimentais?
*   As energias características correspondem às esperadas? Quais os devios percentuais?
*   Os ângulos para n=2 e n=3 estão compatíveis com os esperados?

**Informações para a análise:**

*   A detecção de raio X obedece uma distribuição de Poisson. Por isso a incerteza é dada por σ(**N**)=$\sqrt{N}$/Δt, onde **N** é a soma dos fótons no intervalo de tempo.
*   Para calcular a energia dos picos característicos primeiro precisamos obter o λ a partir dos valores de θ usando a equação de Bragg **nλ = 2dsen(θ)** , onde d = 0.282 nm, que é o *d* do NaCl e podemos usar n=1 para encontrar o valor de λ. Para encontrar o valor da energia fazemos E = hc/λ, onde: 
  * **h = $4.14 × 10^{-15}$ eV s**  
  * **c = $3 × 10^{8}$ m/s**

* Para verificar os ângulos para n=2 e n=3 calculamos os valores teóricos usando o λ encontrado anterioemente para n=1 e comparamos com os valores dos gráficos.
* Os valores teóricos para kα e kβ do Mo são:
  * kα = 17.44 keV
  * kβ = 19.60 keV





In [ ]:
#Juntando os 3 subconjuntos do conjunto 1:

merge1c1 = pd.merge(c1_1, c1_2, how='outer', on = '\u03B8 (°)')
c1 = pd.merge(merge1c1, c1_3, how='outer', on = '\u03B8 (°)')
c1.columns = ['\u03B8 (°)', 'Count 1', 'Count 2', 'Count 3']

#Somando as contagens dos 3 subconjuntos:
c1['Count Total'] = c1['Count 1'] + c1['Count 2'] + c1['Count 3']
c1['σ'] = np.sqrt(c1['Count Total'])


In [ ]:
#Plotting os 3 subconjuntos do conjuto 1:

p1 = go.Scatter(x=c1['\u03B8 (°)'], y=c1['Count 1'], mode='lines', name='Count 1')
p2 = go.Scatter(x=c1['\u03B8 (°)'], y=c1['Count 2'], mode='lines', name='Count 2')
p3 = go.Scatter(x=c1['\u03B8 (°)'], y=c1['Count 3'], mode='lines', name='Count 3')

plots1 = [p1,p2,p3]
layout1 = go.Layout(title='Detecção de Raio X - Conjunto 1 de dados')
figure1 = go.Figure(data=plots1, layout=layout1)
figure1



In [ ]:
#Plotting a soma das contagens:

figure2 = px.line(c1, x='\u03B8 (°)', y='Count Total', title = 'Detecção de Raio X - Conjunto 1 de dados com a soma da intensidade', error_y=c1['σ'])
figure2

In [ ]:
#Selecionando os picos para ajuste da gaussiana onde n=1 (os 2 maiores picos):

c1x_n1 = c1['θ (°)']
c1y_n1 = c1['Count Total']
c1_s = c1['σ']

#kbeta para n=1:
c1x_b_n1 = c1x_n1[(c1x_n1 >= 6.1) & (c1x_n1 <= 6.6)] #valores em x no pico kbeta
c1y_b_n1 = c1y_n1[(c1x_n1 >= 6.1) & (c1x_n1 <= 6.6)] #valores em y no pico kbeta
c1_s_b = c1_s[(c1x_n1 >= 6.1) & (c1x_n1 <= 6.6)] #sigma dos valores de contagem no pico kbeta
c1_b_n1 = pd.DataFrame({'x': c1x_b_n1,'y': c1y_b_n1, 'sy':c1_s_b})

#kalpha para n=1:
c1x_a_n1 = c1x_n1[(c1x_n1 >= 6.9) & (c1x_n1 <= 7.6)] #valores em x no pico kalpha
c1y_a_n1 = c1y_n1[(c1x_n1 >= 6.9) & (c1x_n1 <= 7.6)] #valores em y no pico kalpha
c1_s_a = c1_s[(c1x_n1 >= 6.9) & (c1x_n1 <= 7.6)] #sigma dos valores de contagem no pico kalpha
c1_a_n1 = pd.DataFrame({'x': c1x_a_n1,'y': c1y_a_n1, 'sy':c1_s_a})

#Usando a função describe para calcular as tendências centrais: (alterar o nome da variável para obter os valores pra kalpha)

c1_b_n1

In [ ]:
!pip install lmfit

##**Ajustando a função usando lmfit:**

* Aqui não foram considerado os valores das incertezas nas contagens.

In [ ]:
#Defininfo a função gaussiana:
from lmfit import Model


def gaussian(x, amp, cen, wid):
    return amp*np.exp(-(x-cen)**2/(2*wid**2))

#Ajuste no pico kbeta, n=1, usando lmfit:
b_gmodel = Model(gaussian)
b_result = b_gmodel.fit(c1_b_n1['y'], x=c1x_b_n1, amp=2700, cen=6.3, wid=0.2)
print(b_result.fit_report())


plt.plot(c1x_b_n1, c1y_b_n1, 'o')
plt.plot(c1x_b_n1, b_result.init_fit, '--', label='initial fit')
plt.plot(c1x_b_n1, b_result.best_fit, '-', label='best fit')
plt.title('Fitting kβ')
plt.legend()
plt.show()


In [ ]:
#Ajuste no pico kalpha, n=1, usando lmfit:

a_gmodel = Model(gaussian)
a_result = a_gmodel.fit(c1y_a_n1, x=c1x_a_n1, amp=5700, cen=7.2, wid=0.1)

print(a_result.fit_report())

plt.plot(c1x_a_n1, c1y_a_n1, 'o')
plt.plot(c1x_a_n1, a_result.init_fit, '--', label='initial fit')
plt.plot(c1x_a_n1, a_result.best_fit, '-', label='best fit')
plt.title('Fitting kα')
plt.legend()
plt.show()

In [ ]:
#Resíduos para kbeta, n=1:

residuos_b1 = c1y_b_n1 - b_result.best_fit
plt.scatter(c1y_b_n1, residuos_b1)
plt.axhline(y=0, c='red', ls='--')
plt.title('Resíduos kβ, n=1')

In [ ]:
#Resíduos para kalpha, n=1:

residuos_a1 = c1y_a_n1 - a_result.best_fit
plt.scatter(c1y_a_n1, residuos_a1)
plt.axhline(y=0, c='red', ls='--')
plt.title('Resíduos kα, n=1')

In [ ]:
#Checando os parâmetros:
a_result.params

##**Ajustando a função usando o curve_fit:**

* Aqui foram usadas as incertezas nas contagens.

In [ ]:
#Fitting com o scipy.optmize.curve_fit:

def gauss (x, a, x0, sigma):  #a=ymax, x0=centro da curva, sigma=largura da curva
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

#Ajuste para kbeta:
popt_b, pcov_b = curve_fit(gauss, c1_b_n1['x'], c1_b_n1['y'], sigma=c1_b_n1['sy'], p0=[2000,6.3,0.2])
a = popt_b[0]
x0 = popt_b[1]
sigma = popt_b[2]
plt.plot(c1_b_n1['x'],c1_b_n1['y'], 'bo', label='Data')
plt.errorbar(c1_b_n1['x'],c1_b_n1['y'],yerr=c1_b_n1['sy'], color='b')
plt.plot(c1_b_n1['x'], gauss(c1_b_n1['x'],*popt_b), color='r', label='Fit:' + '$\mathcal{N}$' + f'$(\mu \\approx {round(x0, 1)}, \sigma \\approx {round(sigma,2)})$')
plt.legend()
print('Params:','\n',popt_b, '\n','\n','Cov matrix:','\n', pcov_b)


In [ ]:
#Ajuste para kalpha:
popt_a, pcov_a = curve_fit(gauss, c1_a_n1['x'], c1_a_n1['y'], sigma=c1_a_n1['sy'], p0=[5000,7.2,0.1])
a = popt_a[0]
x0 = popt_a[1]
sigma = popt_a[2]
plt.plot(c1_a_n1['x'],c1_a_n1['y'], 'go', label='Data')
plt.errorbar(c1_a_n1['x'],c1_a_n1['y'],yerr=c1_a_n1['sy'], color='g')
plt.plot(c1_a_n1['x'], gauss(c1_a_n1['x'],*popt_a), color='purple', label='Fit:' + '$\mathcal{N}$' + f'$(\mu \\approx {round(x0, 1)}, \sigma \\approx {round(sigma,2)})$')
plt.legend()
print('Params:','\n',popt_a, '\n','\n','Cov matrix:','\n', pcov_a)

#**Comparação entre os ajustes:**

In [ ]:
print('Resultado para kβ usando lmfit:','\n')
display(b_result)
print('\n')

print('Resultado para kα usando lmfit:','\n')
display(a_result)
print('\n')

print('Resultado para kβ usando curve_fit:','\n')
print('amp:',popt_b[0],'|','inc:', np.sqrt(pcov_b[0][0]),'|','relative error:',(np.sqrt(pcov_b[0][0])/popt_b[0])*100,'%','\n',
      'cen:',popt_b[1],'|','inc:', np.sqrt(pcov_b[1][1]),'|','relative error:',(np.sqrt(pcov_b[1][1])/popt_b[1])*100,'%','\n',
      'wid:',popt_b[2],'|','inc:', np.sqrt(pcov_b[2][2]),'|','relative error:',(np.sqrt(pcov_b[2][2])/popt_b[2])*100,'%')
print('\n')

print('Resultado para kα usando curve_fit:','\n')
print('amp:',popt_a[0],"|",'inc:', np.sqrt(pcov_a[0][0]),'|','relative error:',(np.sqrt(pcov_a[0][0])/popt_a[0])*100,'%','\n',
      'cen:',popt_a[1],'|','inc:', np.sqrt(pcov_a[1][1]),'|','relative error:',(np.sqrt(pcov_a[1][1])/popt_a[1])*100,'%','\n',
      'wid:',popt_a[2],'|','inc:', np.sqrt(pcov_a[2][2]),'|','relative error:',(np.sqrt(pcov_a[2][2])/popt_a[2])*100,'%')
print('\n')


In [ ]:
#R2:

def rsquared(y, yfit):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = lr(y, yfit)
    return r_value**2

#Valores de R2 para os ajustes feitos com o lmfit:
b_r2_lm = rsquared(c1_b_n1['y'], b_result.best_fit)
a_r2_lm = rsquared(c1_a_n1['y'], a_result.best_fit)

#valores de R2 para os ajustes feitos com o curve_fit():
b_r2_cf = rsquared(c1_b_n1['y'], gauss(c1_b_n1['x'],*popt_b))
a_r2_cf = rsquared(c1_a_n1['y'], gauss(c1_a_n1['x'],*popt_a))

print(a_r2_cf, a_r2_lm, b_r2_cf,b_r2_lm)

##**Usando os valores dos ajustes feitos com lmfit para calcular os valores do objetivo do experiemento:**

* OBS.: Comparando os ajustes podemos ver que não há grandes diferenças entre os valores de ${R^2}$ e os desvios percentuais entre os valores dos parâmetros e suas incertezas.

In [ ]:
#Calculando os valores de λ a partir dos valores de θ ajustados para kaplha e kbeta.
#Os valores de θ estão no parâmetro 'cen' do ajuste, que corresponde à média da gaussiana.
#Vamos usar a equação de Bragg: nλ=2dsen(θ) => λ = 2dsen(θ)

d = 0.282*10**(-9) #metros
#Para kbeta:
lambda_b = 2*d*np.sin(6.3511*(np.pi/180))

#Para kalpha:
lambda_a = 2*d*np.sin(7.18760*(np.pi/180))

lambda_a

In [ ]:
#Calculando os valores da energia a partir dos valores de λ e comparando com seus valores teóricos. A equação usada é E = hc/λ:

h = 4.14*10**(-15) #eV s
c = 3*10**8 #m/s

E_ka_t = 17440 #eV 
E_kb_t = 19600 #eV

#Para kbeta:
E_b = (h*c)/lambda_b
desvio_a = (abs(E_b - E_kb_t)/E_kb_t)*100 #desvio percentual de 0.92%

#Para kalfa:
E_a = (h*c)/lambda_a
desvio_b = (abs(E_a - E_ka_t)/E_ka_t)*100 #desvio percentual de 1.57%


In [ ]:
#Verificando os ângulos para n=2 e n=3 usando os valores de λ encontrados:

#Para kalpha, n=2:
theta_a_n2 = np.arcsin(lambda_a/d)*(180/np.pi) #em graus

#Para kbeta, n=2:
theta_b_n2 = np.arcsin(lambda_b/d)*(180/np.pi) #em graus

#Para kalpha, n=3:
theta_a_n3 = np.arcsin((3*lambda_a)/(2*d))*(180/np.pi) #em graus

#Para kbeta, n=3:
theta_b_n3 = np.arcsin((3*lambda_b)/(2*d))*(180/np.pi) #em graus


#**Conjunto de dados 2:**



*   Apresentar os gráficos de Intensidade (número de fótons por unidade
de tempo) em função de energia (keV).
*   Não esqueça de associar uma incerteza a cada medida de contagem.
*   As energias características correspondem às esperadas? Qual(is) o(s)
desvio(s) percentual(is)? Comente.

*   Para cada espectro (tensão de aceleração variando de 35 kV a 18 kV)
determine o valor de λmin (via determinação de θ min ou Emax
experimental).


*   Através de um gráfico de tensão de aceleração versus λ min, determine
a constante de Planck. Compare com o valor esperado e discuta seu
resultado.

**Informações para análise:**

*   A intensidade (contagem) dada pelo software do equipamento é número de fótons por unidade de tempo. Para fazer o gráfico da intensidade em função da energia vamos obter o λ a partir dos valores de θ do gráfico, usando a equação de Bragg nλ = 2dsen(θ), uma vez que estanos considerando apenas n=1. Então a partit desses valores de λ podemos calcular a energia e fazer o gráfico de Intensidade por energia. Para obter a incerteza dos valores da contagem precisamos considerar o calculo comentado anteriormente: multiplicamos os valores de contagem da tabela por Δt para depois fazer $\sqrt{N}$/Δt. As tabelas para Δt = 5s já tiveram suas correções feitas multiplicando os valores de contagem por 5. O plot de Intensidade/unidade de tenpo X Energia deve ser feito com os valores originais fornecidos pelo software, ou seja, dividindo os valores por 5 para ficar contagem/s.
*   A incerteza a ser associada será a que foi informada no item anterior.
*   Para obter a energia característica podemos seguir com o mesmo procedimento do conjunto 1, de ajustar uma gaussiana nos picos para obter a energia.
*   Vamos realizar os dois items acima para todos os valores de tensão, e para cada um desses valores vamos obter um λmin. Para esses λmin vamos obter uma Emax e plotar Emax X λmin.
*   Para descobrir o valor da constante de planck **h** vamos ajustar o gráfico de Emax X λmin.  






